In [1]:
# First let's get the data 
from sklearn.datasets import fetch_20newsgroups
texts = fetch_20newsgroups(subset = 'train')
dir(texts)

['DESCR', 'data', 'filenames', 'target', 'target_names']

In [2]:
# Define  a get minibatch that returns just a little of the data

BATCH_SIZE = 32
def get_batch(data_idx, texts):
    data_x, data_y = [],[]
    for t in range(BATCH_SIZE):
        if data_idx == len(texts.data):
            return [], [], 0
        data_x.append(texts.data[data_idx])
        data_y.append(texts.targe[data_idx])
        data_idx += 1
    return data_x, data_y, data_idx
# start with index 0
data_idx = 0

In [5]:
import numpy as np

from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.linear_model import Perceptron
from sklearn.naive_bayes import MultinomialNB

vectorizer = HashingVectorizer(decode_error = 'ignore', n_features = 2 ** 18)
all_classes = np.array(range(20))

